In [13]:
!pip install -q spinspg spglib pymatgen gemmi  # ≈10 s

In [14]:

import numpy as np, math
from pymatgen.io.cif import CifParser
from pymatgen.core.periodic_table import Element
from pymatgen.core.structure import Magmom          # scalar / vector helper
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
import spglib

def load_mcif(fname):
    """Return lattice (3×3 Å), frac-coords, Z, and magmoms (as 3D vectors in fractional cell coords)."""
    struct = CifParser(fname).parse_structures(primitive=True)[0]
    lat    = struct.lattice.matrix
    # Corrected line to access the element's atomic number
    nums   = np.array([site.species.elements[0].Z for site in struct], int)
    pos    = np.array([site.frac_coords for site in struct])


    # Magnetic moments, as in your previous code
    raw = struct.site_properties.get("magmom", [0.0]*len(struct))
    mags_frac = []
    for m in raw:
        if hasattr(m, "moment"): m = m.moment
        if np.isscalar(m):
            mags_frac.append([0.0, 0.0, 0.0])
        else:
            mags_frac.append(np.asarray(m, float))
    mags = np.array(mags_frac)
    return lat, pos, nums, mags

In [15]:
from spinspg import get_spin_symmetry
import spinspg

def spin_space_group_ops(mcif_path, symprec=1e-5, angle_tol=-1.0):
    lat, pos, nums, mags = load_mcif(mcif_path)
    sog, R, v, S = get_spin_symmetry(
        lat, pos, nums, mags,
        symprec=symprec,
        angle_tolerance=angle_tol,
    )
    if str(sog.spin_only_group_type) == "COPLANAR":
        Z2 = np.eye(3) - 2.0 * np.outer(sog.axis,sog.axis)
        R = np.concatenate((R,R),axis=0)
        v = np.concatenate((v,v),axis=0)
        S = np.concatenate((S,Z2 @ S),axis=0)
    return sog, R, v, S

def magnetic_symmetry_ops(mcif_path):
    """
    Return (S, R, v) for all magnetic symmetry operations.
    """
    lat, pos, nums, mags = load_mcif(mcif_path)
    sym = spglib.get_magnetic_symmetry(
        (lat,pos,nums,mags)
    )
    if sym is None:
        raise RuntimeError("spglib was unable to determine magnetic symmetry.")

    R = sym["rotations"]            # (N,3,3) int
    v = sym["translations"]         # (N,3)   float
    t = sym["time_reversals"]       # (N,)    bool

    # Build S = (-1)^t · det(R) · R  for every operation
    detR = np.round(np.linalg.det(R)).astype(int)   # ±1 for each R
    S =  R.astype(float) * ((-1) ** t)[:,None,None] * detR[:, None, None]

    latinv = np.linalg.inv(lat)

    for i in range(len(S)):
      S[i] = lat @ S[i] @ latinv

    return S, R, v

def little_group(k_frac, lattice, S_all, R_all, v_all,
                 tol=1e-8, return_indices=False):
    """
    Extract the little group of a Bloch wave-vector k from a magnetic
    (spin-space) symmetry list.

    Parameters
    ----------
    k_frac : (3,) array_like
        Wave vector in fractional reciprocal coordinates (units of b1,b2,b3).
    lattice : (3,3) array_like
        Row matrix with direct basis vectors a1,a2,a3 (same one you passed to
        spglib).
    S_all : (N,3,3) ndarray
        Spin-part matrices  (output of `magnetic_symmetry_ops`).
    R_all : (N,3,3) ndarray
        Spatial rotations   (output of `magnetic_symmetry_ops` or spglib).
    v_all : (N,3)   ndarray
        Translations        (output of `magnetic_symmetry_ops` or spglib).
    tol : float, optional
        Equality tolerance for the “k − k' = integer” test (default 1e-8).
    return_indices : bool, optional
        If True, also return the indices of the selected operations inside the
        original arrays.

    Returns
    -------
    S_sub, R_sub, v_sub : ndarrays
        The symmetry operations that form the little group of k, in the same
        order as the originals.
    indices : ndarray  (only if return_indices=True)
        Integer indices of the selected operations.
    """

    #FIXTHIS
    A = np.asarray(lattice, float)
    Ainv = np.linalg.inv(A)
    B = 2.0 * np.pi * Ainv.T          # reciprocal basis as rows
    Binv = A.T / (2.0*np.pi)

    k_cart = B @ np.asarray(k_frac, float)        # k in Cartesian coords

    keep = []
    for i, (S, R) in enumerate(zip(S_all, R_all)):
        sgn = int(round(np.linalg.det(S)))        # ±1  (time reversal & axial)
        # rotation in Cartesian space
        R_cart = A @ R @ Ainv
        k_cart_new = sgn * (R_cart @ k_cart)

        k_frac_new = Binv @ k_cart_new
        diff = k_frac_new - k_frac
        if np.allclose(diff - np.round(diff), 0.0, atol=tol):
            keep.append(i)

    keep = np.array(keep, int)
    S_sub, R_sub, v_sub = S_all[keep], R_all[keep], v_all[keep]

    if return_indices:
        return S_sub, R_sub, v_sub, keep
    return S_sub, R_sub, v_sub

In [16]:
from scipy.linalg import eig

def find_components(char, trivial, P, tol=1e-2):
    dim = np.dot(char,trivial)
    w, V = eig(P)
    keep = np.isclose(w,1.0,atol=tol)
    basis = V[:,keep]

    assert np.allclose(P @ P, P, atol=1e-10), "P is not a projector; check R_cart"

    if not np.allclose(basis.shape[1],dim,atol=1e-2):
      print(P)
      for i in range(len(w)):
        print("eig: ", w[i], "eigw: ", V[:,i])
      #raise RuntimeError("Basis size mismatch, char product: ", dim,"basis size: ",basis.shape[1])

    return basis

In [17]:
def show_ops(R, v, S):
    np.set_printoptions(precision=3, suppress=True)
    print(f"{len(R)} spin-space operations found\n"
          "  S  ||    R    |      v")
    for i in range(len(R)):
      print("#",i+1)
      print("S:")
      print(S[i])
      print("R:")
      print(R[i])
      print("v:")
      print(v[i])
      print("----------------------------------------------------------------------")

In [18]:
sog, R, v, S = spin_space_group_ops("MnTe2.mcif", symprec=1e-3, angle_tol=-1)
print("Spin-only group:", sog)
show_ops(R, v, S)

S, R, v = magnetic_symmetry_ops("MnTe2.mcif")
print("Magnetic symmetry:")
show_ops(R, v, S)

Spin-only group: NONCOPLANAR
24 spin-space operations found
  S  ||    R    |      v
# 1
S:
[[ 1. -0. -0.]
 [ 0.  1. -0.]
 [ 0.  0.  1.]]
R:
[[1 0 0]
 [0 1 0]
 [0 0 1]]
v:
[0. 0. 0.]
----------------------------------------------------------------------
# 2
S:
[[ 1. -0. -0.]
 [ 0.  1. -0.]
 [ 0.  0.  1.]]
R:
[[-1  0  0]
 [ 0 -1  0]
 [ 0  0 -1]]
v:
[0. 0. 0.]
----------------------------------------------------------------------
# 3
S:
[[-1.  0. -0.]
 [-0. -1. -0.]
 [-0. -0.  1.]]
R:
[[-1  0  0]
 [ 0 -1  0]
 [ 0  0  1]]
v:
[0.5 0.  0.5]
----------------------------------------------------------------------
# 4
S:
[[-1.  0. -0.]
 [-0. -1. -0.]
 [-0. -0.  1.]]
R:
[[ 1  0  0]
 [ 0  1  0]
 [ 0  0 -1]]
v:
[0.5 0.  0.5]
----------------------------------------------------------------------
# 5
S:
[[ 1.  0.  0.]
 [ 0. -1.  0.]
 [ 0. -0. -1.]]
R:
[[ 1  0  0]
 [ 0 -1  0]
 [ 0  0 -1]]
v:
[0.5 0.5 1. ]
----------------------------------------------------------------------
# 6
S:
[[ 1.  0.  0.]
 [ 

In [19]:
sog, R, v, S = spin_space_group_ops("Mn3Sn.mcif", symprec=1e-3, angle_tol=-1)


/tmp/ipython-input-14-1129167705.py:10: UserWarning: Issues encountered while parsing CIF: 2 fractional coordinates rounded to ideal values to avoid issues with finite precision.
Skipping relative stoichiometry check because CIF does not contain formula keys.
  struct = CifParser(fname).parse_structures(primitive=True)[0]


In [20]:
def char_hall(S,R):
    return np.trace(R, axis1=-2, axis2=-1) * np.linalg.det(R) * np.linalg.det(S) / np.sqrt(len(R))

def char_S(S,R):
    return np.trace(S, axis1=-2, axis2=-1) / np.sqrt(len(R))

def char_ToddShall(S,R):
    return np.trace(R, axis1=-2, axis2=-1) * np.linalg.det(R) * np.trace(S, axis1=-2, axis2=-1) / np.sqrt(len(R))

def char_spinconductivity(S,R):
    return (np.trace(R, axis1=-2, axis2=-1)**2) * np.trace(S, axis1=-2, axis2=-1) / np.sqrt(len(R))

def triv(S,R):
    return np.ones(R.shape[0]) / np.sqrt(len(R))

In [21]:
from numpy.linalg import matrix_power

def chi(R, p=1):
    """
    χ(R^p)  – trace of the p-th power of R.
    """
    return np.trace(matrix_power(R, p), axis1=-1, axis2=-2)

def chi4(R):
    """
    χ₄(R)  for any square matrix R  (typically a 3×3 rotation).
    """
    t1 = chi(R, 1)                # χ(R)
    t2 = chi(R, 2)                # χ(R²)
    t3 = chi(R, 3)                # χ(R³)
    t4 = chi(R, 4)                # χ(R⁴)

    return (1/24) * (t1**4
                     + 8 * t3 * t1
                     + 3 * t2**2
                     + 6 * t2 * t1**2
                     + 6 * t4)

def chi6(R):
    """
    χ₆(R)  for any square matrix R.
    """
    t1 = chi(R, 1)                # χ(R)
    t2 = chi(R, 2)                # χ(R²)
    t3 = chi(R, 3)                # χ(R³)
    t4 = chi(R, 4)                # χ(R⁴)
    t5 = chi(R, 5)                # χ(R⁵)
    t6 = chi(R, 6)                # χ(R⁶)

    return (1/720) * (
          t1**6
        + 15 * t2 * t1**4
        + 45 * t2**2 * t1**2
        + 15 * t2**3
        + 40 * t3 * t1**3
        + 120 * t3 * t2 * t1
        + 40 * t3**2
        + 90 * t4 * t1**2
        + 90 * t4 * t2
        + 144 * t5 * t1
        + 120 * t6
    )

def chi2(R):
  t1 = chi(R,1)
  t2= chi(R,2)
  return (1/2) * (t1**2 + t2)


def char_4thorder(S,R):
  return chi4(R) * np.trace(S, axis1=-2, axis2=-1) / np.sqrt(len(R))
def char_2ndorder(S,R):
  return chi2(R) * np.trace(S, axis1=-2, axis2=-1) / np.sqrt(len(R))
def char_6thorder(S,R):
  return chi6(R) * np.trace(S, axis1=-2, axis2=-1) / np.sqrt(len(R))

In [22]:
def first_part_calculation(name, Mat, shape, check, cartesian = False, normalize = True, A = np.eye(3), Rot = np.eye(3)):

  if np.array_equal(A, np.eye(3)):
    A = np.eye(Mat.shape[1])
  if np.array_equal(Rot, np.eye(3)):
    Rot = np.eye(Mat.shape[1])
  char = np.trace(Mat, axis1=-2, axis2=-1)/np.sqrt(len(Mat))
  trivial = np.ones(len(Mat))/np.sqrt(len(Mat))
  print(name, ":", np.dot(char,trivial))

  assert np.allclose(np.dot(char,trivial),check, atol=1e-2), "Character mismatch"
  if np.allclose(check,0,atol=1e-4):
    return


  P = np.sum(Mat,axis=0)/len(Mat)
  V = find_components(char,trivial,P)

  if np.allclose(V.imag, 0.0, atol=1e-6):   # or whatever tolerance you trust
    V = V.real

  V[np.abs(V) < 1e-4] = 0.0

  for i in range(len(Mat)):
    assert np.allclose(Mat[i]@V,V), "False invariant component"

  if cartesian:
    V = np.array([ Rot @ A @ V[:,i] for i in range(V.shape[1])]).T



  V[np.abs(V) < 1e-4] = 0.0

  if normalize:
    normalized_V = []
    for i in range(V.shape[1]):
        col = V[:, i]
        non_zero_indices = np.nonzero(col)[0]
        if len(non_zero_indices) > 0:
            min_abs_val = np.min(np.abs(col[non_zero_indices]))
            normalized_V.append(col / min_abs_val)
        else:
            normalized_V.append(col) # Handle columns with all zeros
    V = np.array(normalized_V).T


  V = np.array([V[:,i].reshape(shape) for i in range(V.shape[1])])

  print(name, " basis:")
  for i in range(len(V)):
    print("#",i+1,":")
    print(V[i])



  return


def first_part(S,R, cartesian = False, normalize = True, A = np.eye(3), Rot = np.eye(3)):

  Mat = np.linalg.det(R)[:,None, None]*np.linalg.det(S)[:,None,None]*R
  first_part_calculation("Hall", Mat, (3), np.dot(char_hall(S,R),triv(S,R)), cartesian, normalize, A,Rot=Rot)

  Mat = S
  first_part_calculation("M", Mat, (3), np.dot(char_S(S,R),triv(S,R)), cartesian, normalize,Rot=Rot)

  Mat = np.array([np.kron(S[i],np.linalg.det(R[i]) * R[i]) for i in range(len(R))])
  first_part_calculation("Transverse spin conductivity", Mat, (3,3), np.dot(char_ToddShall(S,R),triv(S,R)), cartesian, normalize, np.kron(np.eye(3),A),Rot=np.kron(Rot,Rot))

  Mat = np.array([np.kron(S[i],R[i]) for i in range(len(R))])
  Mat = np.array([np.kron(Mat[i],R[i]) for i in range(len(R))])
  first_part_calculation("Spin conductivity", Mat, (3,3,3), np.dot(char_spinconductivity(S,R),triv(S,R)), cartesian, normalize, np.kron(np.kron(np.eye(3),A),A),Rot=np.kron(Rot,np.kron(Rot,Rot)))
  print("Second order spin splittings:")
  print(np.dot(char_2ndorder(S,R),triv(S,R)))
  print("Fourth order spin splittings:")
  print(np.dot(char_4thorder(S,R),triv(S,R)))
  print("Sixth order spin splittings:")
  print(np.dot(char_6thorder(S,R),triv(S,R)))



In [23]:
filepath = "Mn3Ir.mcif"
cartesian = True
normalize = True

Rot = np.array([[  0.7886751, -0.2113249, -0.5773503],
  [-0.2113249,  0.7886751, -0.5773503],
   [0.5773503,  0.5773503,  0.5773503 ]])

Rot = np.eye(3)

print(Rot @ np.array([1,1,1]))


soc, R, v, S = spin_space_group_ops(filepath, symprec=1e-3, angle_tol=-1)
lat, pos, nums, mags = load_mcif(filepath)
A=lat

print(filepath)
print("No SOC")
first_part(S,R, cartesian=cartesian, normalize=normalize, A = A, Rot=Rot)

S, R, v = magnetic_symmetry_ops(filepath)

print("SOC")
first_part(S,R, cartesian = cartesian, normalize=normalize, A=A, Rot=Rot)


[1. 1. 1.]
Mn3Ir.mcif
No SOC
Hall : -1.5594808180335136e-18
M : 1.663908511416508e-16
Transverse spin conductivity : 0.0
Spin conductivity : 1.0000000000000004
Spin conductivity  basis:
# 1 :
[[[ 2.  0.  0.]
  [ 0. -1.  0.]
  [ 0.  0. -1.]]

 [[-1.  0.  0.]
  [ 0.  2.  0.]
  [ 0.  0. -1.]]

 [[-1.  0.  0.]
  [ 0. -1.  0.]
  [ 0.  0.  2.]]]
Second order spin splittings:
1.0000000000000004
Fourth order spin splittings:
2.000000000000001
Sixth order spin splittings:
3.0000000000000013
SOC
Hall : 1.0000000000000002
Hall  basis:
# 1 :
[-1. -1. -1.]
M : 1.0000000000000002
M  basis:
# 1 :
[-1. -1. -1.]
Transverse spin conductivity : 1.0000000000000007
Transverse spin conductivity  basis:
# 1 :
[[ 0.  1. -1.]
 [-1.  0.  1.]
 [ 1. -1.  0.]]
Spin conductivity : 4.999999999999999
Spin conductivity  basis:
# 1 :
[[[1. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 1. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 1.]]]
# 2 :
[[[-1.    -2.748 -2.748]
  [ 0.     0.     0.   ]
  [ 0.   

In [43]:
filepath = "Mn3Ge.mcif"
cartesian = False
normalize = True


soc, R, v, S = spin_space_group_ops(filepath, symprec=1e-3, angle_tol=-1)
lat, pos, nums, mags = load_mcif(filepath)
A=lat

print(A)

print(filepath)
print("No SOC")
first_part(S,R, cartesian=cartesian, normalize=normalize, A = A)

S, R, v = magnetic_symmetry_ops(filepath)

print("SOC")
first_part(S,R, cartesian = cartesian, normalize=normalize, A=A)


[[ 5.302  0.     0.   ]
 [-2.651  4.592  0.   ]
 [ 0.     0.     4.289]]
Mn3Ge.mcif
No SOC
Hall : -6.938893903907228e-18
M : 2.5673907444456745e-16
Transverse spin conductivity : 2.7755575615628914e-16
Spin conductivity : 1.0000000000000013
Spin conductivity  basis:
# 1 :
[[[ 0.    -1.732  0.   ]
  [-1.732 -1.732  0.   ]
  [ 0.     0.     0.   ]]

 [[ 2.     1.     0.   ]
  [ 1.    -1.     0.   ]
  [ 0.     0.     0.   ]]

 [[ 0.     0.     0.   ]
  [ 0.     0.     0.   ]
  [ 0.     0.     0.   ]]]
Second order spin splittings:
1.0000000000000013
Fourth order spin splittings:
3.0000000000000036
Sixth order spin splittings:
5.000000000000006
SOC
Hall : 0.9999999999999999
Hall  basis:
# 1 :
[ 1. -1.  0.]
M : 0.9999999999999999
M  basis:
# 1 :
[ 1.    -1.366  0.   ]
Transverse spin conductivity : 1.9999999999999996
Transverse spin conductivity  basis:
# 1 :
[[0.    0.    2.732]
 [0.    0.    1.   ]
 [0.    0.    0.   ]]
# 2 :
[[0. 0. 0.]
 [0. 0. 0.]
 [1. 1. 0.]]
Spin conductivity : 6.9999

/tmp/ipython-input-14-1129167705.py:10: UserWarning: Issues encountered while parsing CIF: 2 fractional coordinates rounded to ideal values to avoid issues with finite precision.
Skipping relative stoichiometry check because CIF does not contain formula keys.
  struct = CifParser(fname).parse_structures(primitive=True)[0]


In [25]:
filepath = "MnTe2.mcif"
cartesian = False
normalize = True


soc, R, v, S = spin_space_group_ops(filepath, symprec=1e-3, angle_tol=-1)
lat, pos, nums, mags = load_mcif(filepath)
A=lat

print(filepath)
print("No SOC")
first_part(S,R, cartesian=cartesian, normalize=normalize, A = A)

S, R, v = magnetic_symmetry_ops(filepath)

print("SOC")
first_part(S,R, cartesian = cartesian, normalize=normalize, A=A)


MnTe2.mcif
No SOC
Hall : -8.326672684688674e-17
M : -4.9135433675615493e-17
Transverse spin conductivity : 1.0
Transverse spin conductivity  basis:
# 1 :
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
Spin conductivity : 1.9999999999999998
Spin conductivity  basis:
# 1 :
[[[ 0.  0.  0.]
  [ 0.  0. -1.]
  [ 0.  0.  0.]]

 [[ 0.  0.  0.]
  [ 0.  0.  0.]
  [-1.  0.  0.]]

 [[ 0. -1.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]]
# 2 :
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 1. 0.]]

 [[0. 0. 1.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [1. 0. 0.]
  [0. 0. 0.]]]
Second order spin splittings:
0.9999999999999998
Fourth order spin splittings:
2.9999999999999996
Sixth order spin splittings:
6.0
SOC
Hall : -2.7755575615628914e-17
M : -2.7755575615628907e-17
Transverse spin conductivity : 1.0000000000000002
Transverse spin conductivity  basis:
# 1 :
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
Spin conductivity : 2.0000000000000004
Spin conductivity  basis:
# 1 :
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 1. 0.]]

 [[0.

In [40]:
filepath = "Mn3Sn.mcif"
cartesian = False
normalize = True


soc, R, v, S = spin_space_group_ops(filepath, symprec=1e-3, angle_tol=-1)
lat, pos, nums, mags = load_mcif(filepath)
A=lat
print(A)

print(filepath)
print("No SOC")
first_part(S,R, cartesian=cartesian, normalize=normalize, A = A)

S, R, v = magnetic_symmetry_ops(filepath)

print("SOC")
first_part(S,R, cartesian = cartesian, normalize=normalize, A=A)


[[ 5.665  0.     0.   ]
 [-2.832  4.906  0.   ]
 [ 0.     0.     4.531]]
Mn3Sn.mcif
No SOC
Hall : 6.938893903907228e-18
M : -1.3183898417423734e-16
Transverse spin conductivity : 8.326672684688674e-17
Spin conductivity : 1.0
Spin conductivity  basis:
# 1 :
[[[-1.    -2.     0.   ]
  [-2.    -1.     0.   ]
  [ 0.     0.     0.   ]]

 [[ 1.732  0.     0.   ]
  [ 0.    -1.732  0.   ]
  [ 0.     0.     0.   ]]

 [[ 0.     0.     0.   ]
  [ 0.     0.     0.   ]
  [ 0.     0.     0.   ]]]
Second order spin splittings:
0.9999999999999998
Fourth order spin splittings:
3.0
Sixth order spin splittings:
5.0
SOC
Hall : 0.9999999999999997
Hall  basis:
# 1 :
[0. 1. 0.]
M : 0.9999999999999999
M  basis:
# 1 :
[0. 1. 0.]
Transverse spin conductivity : 1.9999999999999996
Transverse spin conductivity  basis:
# 1 :
[[ 0.     0.    14.928]
 [ 0.     0.    -1.   ]
 [ 0.     0.     0.   ]]
# 2 :
[[0. 0. 0.]
 [0. 0. 0.]
 [2. 1. 0.]]
Spin conductivity : 6.999999999999998
Spin conductivity  basis:
# 1 :
[[[0. 0

/tmp/ipython-input-14-1129167705.py:10: UserWarning: Issues encountered while parsing CIF: 2 fractional coordinates rounded to ideal values to avoid issues with finite precision.
Skipping relative stoichiometry check because CIF does not contain formula keys.
  struct = CifParser(fname).parse_structures(primitive=True)[0]


In [39]:
filepath = "0.124_Mn3NiN.mcif"
cartesian = False
normalize = False


Rot = np.array([[  0.9773032,  0.1380138,  0.1607192],
   [0.1380138,  0.1607718, -0.9772946],
  [-0.1607192,  0.9772946,  0.1380750]]).T



soc, R, v, S = spin_space_group_ops(filepath, symprec=1e-3, angle_tol=-1)
lat, pos, nums, mags = load_mcif(filepath)
A=lat

print(A)
A_inv = np.linalg.inv(A)
S = np.array([A_inv @ S[i] @ A for i in range(len(S))])

print(filepath)
print("No SOC")
first_part(S,R, cartesian=cartesian, normalize=normalize, A = A, Rot=Rot)

S, R, v = magnetic_symmetry_ops(filepath)

print("SOC")
first_part(S,R, cartesian = cartesian, normalize=normalize, A=A, Rot=Rot)


/usr/local/lib/python3.11/dist-packages/pymatgen/io/cif.py:1313: UserWarning: Incorrect stoichiometry:
  CIF={'Mn': 3.0, 'Ni': 1.0, 'N': 1.0}
  PMG={'Mn': 3.0, 'N': 0.96, 'Ni': 0.9}
  ratios={'Ni': 0.9, 'N': 0.96, 'Mn': 1.0}
  if struct := self._get_structure(data, primitive, symmetrized, check_occu=check_occu):


[[ 2.749  1.587  2.244]
 [ 2.749 -1.587 -2.244]
 [ 0.     3.174 -2.244]]
0.124_Mn3NiN.mcif
No SOC
Hall : -1.1967821673894356e-17
M : -1.185475085351195e-16
Transverse spin conductivity : 5.551115123125783e-17
Spin conductivity : 1.0
Spin conductivity  basis:
# 1 :
[[[ 0.577  0.     0.   ]
  [ 0.    -0.315  0.   ]
  [ 0.     0.    -0.261]]

 [[-0.018  0.     0.   ]
  [ 0.    -0.279  0.   ]
  [ 0.     0.     0.297]]

 [[-0.025  0.     0.   ]
  [ 0.    -0.395  0.   ]
  [ 0.     0.     0.42 ]]]
Second order spin splittings:
0.9999999999999999
Fourth order spin splittings:
2.0
Sixth order spin splittings:
3.0
SOC
Hall : 1.0
Hall  basis:
# 1 :
[-0.577  0.577  0.577]
M : 1.0000000000000002
M  basis:
# 1 :
[ 0.161 -0.977  0.138]
Transverse spin conductivity : 3.0000000000000004
Transverse spin conductivity  basis:
# 1 :
[[ 0.577  0.266  0.266]
 [ 0.     0.205  0.067]
 [ 0.     0.469 -0.508]]
# 2 :
[[ 0.065 -0.454  0.362]
 [-0.249  0.288  0.416]
 [ 0.517  0.212  0.171]]
# 3 :
[[-0.077  0.446 -0